In [1]:
#!/usr/local/bin/python

import os, sys
import json
import numpy as np
import matplotlib.pyplot as plt
import chainer
import subprocess
from chainer import cuda
from matplotlib import animation
from optparse import OptionParser

from elecpy.solver.PDE import PDE
from elecpy.stim.ExtracellularStimulator import ExtracellularStimulator
from elecpy.stim.MembraneStimulator import MembraneStimulator
from elecpy.cell.ohararudy.model import model as cell_model_ohararudy
from elecpy.cell.luorudy.model import model as cell_model_luorudy
from elecpy.cell.mahajan.model import model as cell_model_mahajan
from elecpy.util.cmap_bipolar import bipolar
import elecpy.elecpy as elp

from matplotlib import animation, rc
from IPython.display import HTML

import time
import cv2

xp = cuda.cupy

In [2]:
def conv_cntSave2time(cnt_save):
    udt          = sim_params['time']['udt']     # Universal time step (ms)
    cnt_log      = sim_params['log']['cnt']      # num of udt for logging
    return udt*cnt_log

def conv_cntUdt2time(cnt_udt):
    udt          = sim_params['time']['udt']     # Universal time step (ms)
    return cnt_udt * udt

def conv_time2cntUdt(t):
    udt          = sim_params['time']['udt']     # Universal time step (ms)
    return int(t/udt)

def conv_time2cntSave(t):
    udt          = sim_params['time']['udt']     # Universal time step (ms)
    cnt_log      = sim_params['log']['cnt']      # num of udt for logging
    return conv_time2cntUdt(t) // cnt_log 

In [3]:
sim_parent = '/mnt/Omer/Project/03.LinearRegionalCooling/SimulationResult/2D/'
directory_template = '20180820-%s'
device_temps = [-40]
temperatures = [9, 6, 3]
widths = [5, 3, 1]
times = [920, 930, 955, 965]
num = 1

alpha = 0.0000001466 #水の熱拡散係数
rho_c = 4200000
cell_height = 0.00015
h = 290 #W/(m^2 K)
dt = 0.001
dx = 0.00015

small_mat = np.identity(200)*(-4)+np.tri(200,200,-1)-np.tri(200,200,-2)-np.tri(200,200,0)+np.tri(200,200,1)
def_mat = np.zeros((200*200, 200*200))
for i in range(200):
    for j in range(200):
        if i == j:
            def_mat[200*i:200*(i+1), 200*j:200*(j+1)] = small_mat
        if i-j == 1 or i-j == -1:
            def_mat[200*i:200*(i+1), 200*j:200*(j+1)] = np.identity(200)

In [5]:
for d_temp in device_temps:
    for temp in temperatures:
        for width in widths:
            for index, time1 in enumerate(times):
                if num < 17:
                    pass
                else:
                    temp_array = (np.ones((200, 200))*310.15).flatten()
                    cooling_flag = 1

                    print(directory_template%num)

                    sim_path = os.path.join(sim_parent, directory_template%num)
                    if not os.path.exists(sim_path):
                        os.makedirs(sim_path)

                    #冷却領域
                    temp_target = np.ones((420, 420)) * 310.15
                    temp_target[210-width:211+width, 210:] = 273.15 + d_temp

                    rows,cols = temp_target.shape
                    M = cv2.getRotationMatrix2D((cols/2,rows/2),45*(index-1),1)
                    dst = cv2.warpAffine(temp_target,M,(cols,rows))
                    temp_target = dst[110:310, 110:310]

                    temp_target = temp_target.flatten()
                    mask = (temp_target < 300)*1

                    #シミュレーション開始
                    with open ('elecpy/temp/sim_params_cooling.json','r') as f:
                        sim_params = json.load(f)
                    sim_params['log']['path'] = sim_path
                    sim_params['restart']['count'] = time1
                    print json.dumps(sim_params, indent=4)

                    with open('{0}/sim_params.json'.format(sim_params['log']['path'] ), 'w') as f:
                        json.dump(sim_params, f, indent=4)

                    assert sim_params is not None

                    cuda.get_device(1).use()

                    # Constants
                    Sv           = 1400                  # Surface-to-volume ratio (cm^-1)
                    Cm           = 1.0                   # Membrane capacitance (uF/cm^2)
                    sigma_l_i    = 1.74                  # (mS/cm)
                    sigma_t_i    = 0.19                  # (mS/cm)
                    sigma_l_e    = 6.25                  # (mS/cm)
                    sigma_t_e    = 2.36                  # (mS/cm)

                    # Geometory settings
                    im_h         = sim_params['geometory']['height']
                    im_w         = sim_params['geometory']['width']
                    ds           = sim_params['geometory']['ds'] # Spatial discretization step (cm)
                    N            = im_h*im_w

                    # Time settings
                    udt          = sim_params['time']['udt']     # Universal time step (ms)
                    time_end     = sim_params['time']['end']

                    # Logging settings
                    cnt_log      = sim_params['log']['cnt']      # num of udt for logging
                    savepath     = sim_params['log']['path']

                    # Cell model settings
                    cells = None
                    if sim_params['cell_type'] == 'ohararudy':
                        cells = cell_model_ohararudy(shape=(N,))
                    if sim_params['cell_type'] == 'luorudy':
                        cells = cell_model_luorudy(shape=(N,))
                    if sim_params['cell_type'] == 'mahajan':
                        cells = cell_model_mahajan(shape=(N,))
                    assert cells is not None

                    print "Stimulation settings",
                    stims_ext = []
                    stims_mem = []
                    if 'stimulation' in sim_params.keys():
                        stim_param = sim_params['stimulation']
                        if 'extracellular' in stim_param:
                            for param in stim_param['extracellular']:
                                stim = ExtracellularStimulator(**param)
                                assert tuple(stim.shape) == (im_h, im_w)
                                stims_ext.append(stim)
                        if 'membrane' in stim_param:
                            for param in stim_param['membrane']:
                                stim = MembraneStimulator(**param)
                                assert tuple(stim.shape) == (im_h, im_w)
                                stims_mem.append(stim)
                    print "...done"

                    print "Allocating data...",
                    cells.create()

                    i_ion              = np.zeros((N),dtype=np.float64)
                    phie               = np.zeros((N),dtype=np.float64)
                    i_ext_e            = np.zeros((N),dtype=np.float64)
                    i_ext_i            = np.zeros((N),dtype=np.float64)
                    rhs_phie           = np.zeros((N),dtype=np.float64)
                    rhs_vmem           = np.zeros((N),dtype=np.float64)
                    vmem               = np.copy(cells.get_param('v'))
                    print "...done"

                    print "Initializing data...",
                    if 'restart' in sim_params.keys():
                        cnt_restart = sim_params['restart']['count']
                        srcpath = sim_params['restart']['source']
                        pfx = '_{0:0>4}'.format(cnt_restart)
                        phie = np.load('{0}/phie{1}.npy'.format(srcpath,pfx)).flatten()
                        vmem = np.load('{0}/vmem{1}.npy'.format(srcpath,pfx)).flatten()
                        cells.load('{0}/cell{1}'.format(srcpath,pfx))
                        cnt_udt = cnt_restart * cnt_log
                    print "...done"

                    print 'Building PDE system ...',
                    sigma_l      = sigma_l_e + sigma_l_i
                    sigma_t      = sigma_t_e + sigma_t_i
                    pde_i = PDE( im_h, im_w, sigma_l_i, sigma_t_i, ds, 310.15, xp.asnumpy(cells.state[1].reshape((200,200))))
                    pde_m = PDE( im_h, im_w, sigma_l,   sigma_t,   ds, 310.15, xp.asnumpy(cells.state[1].reshape((200,200))))
                    print '...done'

                    lut_dstep = [
                    (0, 5),
                    (0, 1)
                    ]

                    # Initialization
                    t         = 0.                       # Time (ms)
                    cnt_udt   = 0                        # Count of udt
                    dstep     = 1                        # Time step (# of udt)
                    cnt_save  = -1

                    run_udt   = True                     # Flag of running sim in udt
                    flg_st    = False                    # Flaf of stimulation
                    cnt_st_off = 0

                    print 'Main loop start!'
                    sim_result_image = []
                    start = time.time()

                    while t < time_end:                
                        t = conv_cntUdt2time(cnt_udt)
                        dt = dstep * udt

                        # Stimulation control
                        i_ext_e[:] = 0.0
                        flg_st_temp = False
                        for s in stims_ext:
                            i_ext_e += s.get_current(t)*Sv
                            flg_st_temp = flg_st_temp or s.get_flag(t)
                        for s in stims_mem:
                            cells.set_param('st', s.get_current(t)) 

                        # step.1 cell state transition
                        cells.set_param('dt', dt )
                        cells.set_param('v', vmem )
                        cells.update()
                        i_ion = cells.get_param('it')

                        # step.2 phie (3.4.2)
                        rhs_phie = i_ext_e - i_ext_i - pde_i.forward(vmem)
                        pde_cnt, phie = pde_m.solve(phie, rhs_phie, tol=1e-2, maxcnt=1e5)
                        phie -= phie[0]

                        # step.3 vmem　(3.4.3)
                        rhs_vmem = pde_i.forward(vmem)
                        rhs_vmem += pde_i.forward(phie)
                        rhs_vmem -= i_ion * Sv
                        rhs_vmem += i_ext_i
                        rhs_vmem *= 1 / (Cm * Sv)
                        vmem += dt * rhs_vmem

                        # Logging & error check
                        cnt_save_now = conv_time2cntSave(t)
                        if cnt_save_now != cnt_save:
                            print '------------------{0}ms({1})'.format(t, pde_cnt)
                            cnt_save = cnt_save_now
                            np.save('{0}/phie_{1:0>4}'.format(savepath,cnt_save), phie.reshape((im_h, im_w)))
                            np.save('{0}/vmem_{1:0>4}'.format(savepath,cnt_save), vmem.reshape((im_h, im_w)))
                            cells.save('{0}/cell_{1:0>4}'.format(savepath,cnt_save))
                            sim_result_image.append((t, np.copy(vmem.reshape(im_h, im_w))))

                            if np.min(temp_array) < 310.15-temp:
                                cooling_flag = 0
                            if cooling_flag == 1:
                                temp_array = temp_array + alpha*dt/dx/dx*np.dot(def_mat, temp_array) + dt/rho_c/cell_height*h*mask*(temp_target-temp_array)
                            else:
                                temp_array = temp_array + alpha*dt/dx/dx*np.dot(def_mat, temp_array)
                            temp_array = temp_array.reshape((200, 200))
                            temp_array[0,:] = 310.15
                            temp_array[-1,:] = 310.15
                            temp_array[:,0] = 310.15
                            temp_array[:, -1] = 310.15

                            cells.state[1] = xp.asarray(temp_array.flatten())

                            pde_i = PDE( im_h, im_w, sigma_l_i, sigma_t_i, ds, 310.15, xp.asnumpy(cells.state[1].reshape((200,200))))
                            pde_m = PDE( im_h, im_w, sigma_l,   sigma_t,   ds, 310.15, xp.asnumpy(cells.state[1].reshape((200,200))))

                            temp_array = temp_array.flatten()

                            flg = False
                            for i,v in enumerate(vmem):
                                if v != v :
                                    print "error : invalid value {1} @ {0} ms, index {2}".format(t, v, i)
                                    flg = True
                                    break
                            if flg is True:
                                break

                        # Time srep control
                        for thre, step in lut_dstep:
                            if pde_cnt > thre: dstep = step

                        cnt_udt += dstep

                    print "elecpy done"
                    print "elapsed time:", time.time() - start
        #             np.save(os.path.join(sim_path, 'temp.npy'), temp_array)
                num += 1

 20180820-17
{
    "cell_type": "luorudy", 
    "log": {
        "path": "/mnt/Omer/Project/03.LinearRegionalCooling/SimulationResult/2D/20180820-17", 
        "cnt": 1000
    }, 
    "stimulation": {
        "extracellular": [], 
        "membrane": []
    }, 
    "time": {
        "end": 4000, 
        "udt": 0.001
    }, 
    "geometory": {
        "width": 200, 
        "ds": 0.015, 
        "height": 200
    }, 
    "restart": {
        "count": 920, 
        "source": "/mnt/recordings/SimulationResults/2D/20180718-2"
    }
}
Stimulation settings ...done
Allocating data... ...done
Initializing data... ...done
Building PDE system ...

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBAPRO_CUDA_DRIVER
with the file path of the CUDA driver shared library.
: